In [1]:
import numpy as np
import pandas as pd

---
### Setting up basic matrices:
The Pauli matrices are the four 2x2 matrices, I, X, Y, Z.
For this example, we have a 4x4 Hamiltonian that is decomposed into IZ, ZI, XX, YY, and ZZ terms (this notation is not matrix multiplication--it is a Kronecker product, which takes two 2x2 matrices and gives a 4x4 matrix).

In [2]:
I = np.eye(2)
X = np.array([[0., 1.],
              [1., 0.]])
Y = np.array([[0., -1j],
              [1j, 0.]])
Z = np.array([[1., 0.],
              [0., -1.]])

In [3]:
IZ = np.kron(I, Z)
print(IZ)

[[ 1.  0.  0.  0.]
 [ 0. -1.  0. -0.]
 [ 0.  0.  1.  0.]
 [ 0. -0.  0. -1.]]


In [4]:
ZI = np.kron(Z, I)
print(ZI)

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1. -0.]
 [ 0.  0. -0. -1.]]


In [5]:
XX = np.kron(X, X)
print(XX)

[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


In [6]:
YY = np.kron(Y, Y)
print(YY)

[[ 0.+0.j  0.-0.j  0.-0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  1.-0.j  0.-0.j]
 [ 0.+0.j  1.-0.j  0.+0.j  0.-0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]]


In [7]:
ZZ = np.kron(Z, Z)
print(ZZ)

[[ 1.  0.  0.  0.]
 [ 0. -1.  0. -0.]
 [ 0.  0. -1. -0.]
 [ 0. -0. -0.  1.]]


----
### Hamiltonian
We consider the Hamiltonian:
$H = IZ + ZI - XX - YY + ZZ$

In [8]:
H = IZ + ZI - XX - YY + ZZ
print(H)

[[ 3.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j -2.+0.j  0.+0.j]
 [ 0.+0.j -2.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]]


---
### Measuring an observable
An observable is a matrix that we can measure against a state $\vec{\psi}$. The Hamiltonian is an observable. So are the Pauli terms that it is written as a sum of.

To measure some observable matrix $M$ with respect to a state $\vec{\psi}$, we just compute $\vec{\psi}^{\dagger} M \vec{\psi}$. The $\dagger$ means conjugate transpose = transpose the matrix and then take complex conjugates. In this specific example, all the numbers are real, so it happens to be the same as an ordinary matrix transpose.

Here is a helper method that does everything for us:

In [9]:
def expected_value(M, state):
    return np.asmatrix(state).getH() @ M @ state  # .getH() does conjugate tranpose

---
### Let's try for $\vec{\psi} = [0, 1, 0, 0]^T$

In [10]:
state = np.array([[0.], [1.], [0.], [0.]])

#### Method 1: measure $H$ directly
A quantum computer can't actually measure $H$ directly, but let's just check what the answer should be.

In [11]:
print(expected_value(H, state))

[[-1.+0.j]]


#### Method 2: linearity of expectation
A quantum computer can compute the Pauli observables though. So let's use linearity of expectation instead:

In [12]:
IZ_expectation = expected_value(IZ, state)
ZI_expectation = expected_value(ZI, state)
XX_expectation = expected_value(XX, state)
YY_expectation = expected_value(YY, state)
ZZ_expectation = expected_value(ZZ, state)
print(IZ_expectation, ZI_expectation, XX_expectation, YY_expectation, ZZ_expectation)
print(IZ_expectation + ZI_expectation - XX_expectation - YY_expectation + ZZ_expectation)

[[-1.]] [[1.]] [[0.]] [[0.+0.j]] [[-1.]]
[[-1.+0.j]]


Yup, we got -1 both ways!

---
### Variances / Covariances

In [13]:
def variance(M, state):
    # variances is <M^2> - <M>^2
    return expected_value(M @ M, state) - expected_value(M, state) ** 2

def covariance(M_1, M_2, state):
    return expected_value(M_1 @ M_2, state) - expected_value(M_1, state) * expected_value(M_2, state)

The cells below show that:
- Var(H) = 4
- Var(IZ) + Var(ZI) + Var(XX) + Var(YY) + Var(ZZ) = 2 -- not equal because we need to account for covariances
- the summed covariances do indeed equal 4. Note that the XX and ZZ

In [14]:
variance(H, state)

matrix([[4.+0.j]])

In [15]:
variance(IZ, state)

matrix([[0.]])

In [16]:
variance(ZI, state)

matrix([[0.]])

In [17]:
variance(XX, state)

matrix([[1.]])

In [18]:
variance(YY, state)

matrix([[1.+0.j]])

In [19]:
variance(ZZ, state)

matrix([[0.]])

In [20]:
terms, coefficients = [IZ, ZI, XX, YY, ZZ], [1., 1., -1., -1., 1.]
total = np.array([[0.+0.j]])
for i in range(5):
    for j in range(5):
        cov = covariance(terms[i], terms[j], state)
        if cov != 0:
            print(cov, i, j)
            total += (coefficients[i] * coefficients[j] * covariance(terms[i], terms[j], state))
print('Variance from summed covariances is %s' % total)

[[1.]] 2 2
[[1.+0.j]] 2 3
[[1.+0.j]] 3 2
[[1.+0.j]] 3 3
Variance from summed covariances is [[4.+0.j]]


----
### Different State, $\vec{\psi} = [0.43 - 0.16i, -0.49, 0.44i, -0.39 + 0.46i]$

In [164]:
state = np.array([[0.43 - 0.16j], [-0.49], [0.43j], [-0.39 + 0.46j]])

In [22]:
variance(H, state)

matrix([[4.35848816-2.22044605e-16j]])

In [23]:
variance(IZ, state), variance(ZI, state), variance(-XX, state), variance(-YY, state), variance(ZZ, state)

(matrix([[0.95576944+0.j]]),
 matrix([[0.989596+0.j]]),
 matrix([[0.76629724+6.6974204e-18j]]),
 matrix([[0.76629724+0.j]]),
 matrix([[0.97693936+0.j]]))

In [24]:
print(covariance(IZ, ZI, state))
print(covariance(IZ, ZZ, state))
print(covariance(ZI, ZZ, state))
print(covariance(-XX, -YY, state))
print(covariance(-XX, ZZ, state))
print(covariance(-YY, ZZ, state))

[[0.1287768+0.j]]
[[-0.06690672+0.j]]
[[-0.1937784+0.j]]
[[0.08370276-3.3487102e-18j]]
[[0.41059608+1.03528297e-18j]]
[[-0.41059608+6.9388939e-18j]]


In [25]:
# Setting 1 (measure all terms separately) would require a number of state preparations equal to:
5 * (0.95576944 + 0.989596 + 0.76629724 + 0.76629724 + 0.97693936)

22.2744964

In [26]:
# Setting 2 (measure {XX}, {YY, ZZ}, {IZ, ZI}) would require a number of state preparations equal to:
3 * (0.95576944 + 0.989596 + 0.76629724 + 0.76629724 + 0.97693936 + 2*0.08370276 + 2*0.1287768)

14.6395752

In [27]:
# Setting 3 (measure {XX, YY, ZZ}, {IZ, ZI}) would require a number of state preparations equal to:
2 * (0.95576944 + 0.989596 + 0.76629724 + 0.76629724 + 0.97693936 + 2*0.08370276 + 2*0.41059608 + 2*-0.41059608 + 2*0.1287768)

9.7597168

### Programatically finding optimal groupings

In [28]:
def variance_grouping(grouping, state):
    """
    Computes the variance of a grouping, via the formula
             Var(A+B) = Var(A) + Var(B) + 2*Cov(A,B)
    Inputs:
       Grouping (as a list, e.g. [-XX, -YY, ZZ])
       State (as a np.array, e.g. np.array([[0.], [1.], [0.], [0.]]))
    """
    running_variance = sum([variance(term, state) for term in grouping])
    grouping_size = len(grouping)
    for i in range(grouping_size):
        for j in range(i):
            running_variance += 2*covariance(grouping[i], grouping[j], state)
    return(np.asscalar(running_variance))

In [29]:
# # Credit to: https://stackoverflow.com/questions/19368375/set-partitions-in-python
def partition(collection):
    if len(collection) == 1:
        yield [ collection ]
        return

    first = collection[0]
    for smaller in partition(collection[1:]):
        # insert `first` in each of the subpartition's subsets
        for n, subset in enumerate(smaller):
            yield smaller[:n] + [[ first ] + subset]  + smaller[n+1:]
        # put `first` in its own subset 
        yield [ [ first ] ] + smaller


In [30]:
def grouping_is_pairwise_commuting(grouping_to_test):
    """
    Computes whether all pairs within a grouping commute (in a general sense).
    Inputs:
        grouping_to_test: a single list of *strings*, e.g. ["-XX", "-YY", "ZZ", "IZ", "ZI"]
    Returns:
        True if all pairs commute; False if not all pairs commute
    """
    evaled_grouping_to_test = [eval(term) for term in grouping_to_test]
    running_sum = 0
    for i in range(len(evaled_grouping_to_test)):
        for j in range(i+1, len(evaled_grouping_to_test)):
            # RHS evaluates to 1 if non-commuting, 0 if commuting
            running_sum += not np.array_equal(evaled_grouping_to_test[i] @ evaled_grouping_to_test[j],
                               evaled_grouping_to_test[j] @ evaled_grouping_to_test[i])
    # sum will be 0 iff all pairs commute
    return(running_sum == 0)

def is_valid_partition(partition):
    """
    Computes whether all groupings within a partition are pairwise commuting.
    Inputs:
        partition: a list of lists of strings, e.g. [['-XX', '-YY'], ['ZZ', 'IZ', 'ZI']]
    Outputs:
        1 if all groupings within the partition are pairwise commuting. Else 0.
    """
    return(sum([grouping_is_pairwise_commuting(grouping) for grouping in partition]) == len(partition))

In [66]:
def calculate_expected_state_preps_valid_partitions(term_set, state):
    """
    Computes the expected number of state preparations given every valid partition of a set of terms.
    A valid partition is one in which all groupings are pairwise commuting.
    Inputs:
        term_set: list of *strings*, e.g. ["-XX", "-YY", "ZZ", "IZ", "ZI"]
        state: as a np.array, e.g. np.array([[0.], [1.], [0.], [0.]]))
    """
    evaled_term_set = [eval(term) for term in term_set]
    term_list = []
    expected_state_prep_list = []
    for pp in partition(list(range(len(term_set)))):
         term_list.append([list(np.array(term_set)[grouping]) for grouping in pp])
         expected_state_prep_list.append(sum([variance_grouping(np.array(evaled_term_set)[indices], state)*len(pp) for indices in pp]))
    partition_list = pd.DataFrame({'partition': term_list, 'expected_state_preps': expected_state_prep_list}).sort_values(by=["expected_state_preps"])
    # Eliminate invalid partitions that contain groupings that are non-pairwise-computing.
    return(partition_list[partition_list['partition'].apply(lambda partition: is_valid_partition(partition))])

# Aspuru-Guzik example
test_term_set = ["-XX", "-YY", "ZZ", "IZ", "ZI"]
state = np.array([[0], [1], [0], [0]])

partition_list = calculate_expected_state_preps_valid_partitions(test_term_set, state)
partition_list


,partition,expected_state_preps
7,"[[-XX], [-YY, ZZ], [IZ, ZI]]",(6+0j)
12,"[[-YY], [-XX, ZZ], [IZ, ZI]]",(6+0j)
4,"[[-XX], [-YY], [ZZ, IZ, ZI]]",(6+0j)
48,"[[-YY], [-XX, ZZ], [IZ], [ZI]]",(8+0j)
38,"[[-XX], [-YY, ZZ], [IZ], [ZI]]",(8+0j)
34,"[[-XX], [-YY], [IZ], [ZZ, ZI]]",(8+0j)
14,"[[-XX], [-YY], [ZZ], [IZ, ZI]]",(8+0j)
24,"[[-XX], [-YY], [ZZ, IZ], [ZI]]",(8+0j)
2,"[[-XX, -YY], [ZZ, IZ, ZI]]",(8+0j)
5,"[[-XX, -YY, ZZ], [IZ, ZI]]",(8+0j)


In [46]:
# Helper function: print all pairwise covariances across commuting terms
def pairwise_covariances_commuting_terms(term_set, state):
    covariance_list = []
    evaled_term_set = [eval(term) for term in term_set]
    for i in range(len(evaled_term_set)):
        for j in range(i+1, len(evaled_term_set)):
            # evaluates to 1 if commuting, 0 if non-commuting
            if np.array_equal(evaled_term_set[i] @ evaled_term_set[j],
                               evaled_term_set[j] @ evaled_term_set[i]):
                covariance_list.append([term_set[i], term_set[j], covariance(evaled_term_set[i], evaled_term_set[j], state).item()])
    return(pd.DataFrame(covariance_list, columns=["term 1", "term 2", "covariance"]))

In [47]:
pairwise_covariances_commuting_terms(test_term_set, state)

,term 1,term 2,covariance
0,-XX,-YY,(1+0j)
1,-XX,ZZ,0j
2,-YY,ZZ,0j
3,ZZ,IZ,0j
4,ZZ,ZI,0j
5,IZ,ZI,0j


In [50]:
state_list = [np.array([[1], [0], [0], [0]]),
              np.array([[0], [1], [0], [0]]),
              np.array([[0], [0], [1], [0]]),
              np.array([[0], [0], [0], [1]])]

In [67]:
for state in state_list:
    print(state)
    print(pairwise_covariances_commuting_terms(test_term_set, state))
    print(calculate_expected_state_preps_valid_partitions(test_term_set, state))

[[1]
 [0]
 [0]
 [0]]
  term 1 term 2  covariance
0    -XX    -YY     (-1+0j)
1    -XX     ZZ          0j
2    -YY     ZZ          0j
3     ZZ     IZ          0j
4     ZZ     ZI          0j
5     IZ     ZI          0j
                           partition  expected_state_preps
47    [[-XX, -YY], [ZZ], [IZ], [ZI]]                    0j
35      [[-XX, -YY, ZZ], [IZ], [ZI]]                    0j
31      [[-XX, -YY], [IZ], [ZZ, ZI]]                    0j
21      [[-XX, -YY], [ZZ, IZ], [ZI]]                    0j
11      [[-XX, -YY], [ZZ], [IZ, ZI]]                    0j
5         [[-XX, -YY, ZZ], [IZ, ZI]]                    0j
2         [[-XX, -YY], [ZZ, IZ, ZI]]                    0j
4       [[-XX], [-YY], [ZZ, IZ, ZI]]                (6+0j)
12      [[-YY], [-XX, ZZ], [IZ, ZI]]                (6+0j)
7       [[-XX], [-YY, ZZ], [IZ, ZI]]                (6+0j)
48    [[-YY], [-XX, ZZ], [IZ], [ZI]]                (8+0j)
38    [[-XX], [-YY, ZZ], [IZ], [ZI]]                (8+0j)
24    [[-XX], [-

In [102]:
# ugly code to calculate the expected state preps per each of the basis states
df0 = calculate_expected_state_preps_valid_partitions(test_term_set, state_list[0])
df1 = calculate_expected_state_preps_valid_partitions(test_term_set, state_list[1])
df2 = calculate_expected_state_preps_valid_partitions(test_term_set, state_list[2])
df3 = calculate_expected_state_preps_valid_partitions(test_term_set, state_list[3])

temp = pd.merge(df0, df1, left_index=True, right_index=True)
temp = pd.merge(temp, df2, left_index=True, right_index=True)
temp = pd.merge(temp, df3, left_index=True, right_index=True)
temp = temp.iloc[:,[0,1,3,5,7]]
temp.columns=["partition", "00", "01", "10", "11"]
temp['row_mean'] = temp.iloc[:,1:].mean(axis=1)
temp['row_var'] = temp.iloc[:,1:].var(axis=1)
temp.sort_values(by=["row_mean"])

/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:36: ComplexWarning: Casting complex values to real discards the imaginary part
  return umr_sum(a, axis, dtype, out, keepdims, initial)


,partition,00,01,10,11,row_mean,row_var
5,"[[-XX, -YY, ZZ], [IZ, ZI]]",0j,(8+0j),(8+0j),0j,(4+0j),16.0
2,"[[-XX, -YY], [ZZ, IZ, ZI]]",0j,(8+0j),(8+0j),0j,(4+0j),16.0
35,"[[-XX, -YY, ZZ], [IZ], [ZI]]",0j,(12+0j),(12+0j),0j,(6+0j),36.0
31,"[[-XX, -YY], [IZ], [ZZ, ZI]]",0j,(12+0j),(12+0j),0j,(6+0j),36.0
21,"[[-XX, -YY], [ZZ, IZ], [ZI]]",0j,(12+0j),(12+0j),0j,(6+0j),36.0
11,"[[-XX, -YY], [ZZ], [IZ, ZI]]",0j,(12+0j),(12+0j),0j,(6+0j),36.0
4,"[[-XX], [-YY], [ZZ, IZ, ZI]]",(6+0j),(6+0j),(6+0j),(6+0j),(6+0j),0.0
12,"[[-YY], [-XX, ZZ], [IZ, ZI]]",(6+0j),(6+0j),(6+0j),(6+0j),(6+0j),0.0
7,"[[-XX], [-YY, ZZ], [IZ, ZI]]",(6+0j),(6+0j),(6+0j),(6+0j),(6+0j),0.0
47,"[[-XX, -YY], [ZZ], [IZ], [ZI]]",0j,(16+0j),(16+0j),0j,(8+0j),64.0


In [173]:
# Are the ideal partitions the same with random kets?
import qutip
random_state_list = [qutip.rand_ket(4).full() for i in range(4)]
print(random_state_list)

# ugly code to calculate the expected state preps per each of the basis states
df0 = calculate_expected_state_preps_valid_partitions(test_term_set, random_state_list[0])
df1 = calculate_expected_state_preps_valid_partitions(test_term_set, random_state_list[1])
df2 = calculate_expected_state_preps_valid_partitions(test_term_set, random_state_list[2])
df3 = calculate_expected_state_preps_valid_partitions(test_term_set, random_state_list[3])

temp = pd.merge(df0, df1, left_index=True, right_index=True)
temp = pd.merge(temp, df2, left_index=True, right_index=True)
temp = pd.merge(temp, df3, left_index=True, right_index=True)
temp = temp.iloc[:,[0,1,3,5,7]]
temp.columns=["partition", "00", "01", "10", "11"]
temp['row_mean'] = temp.iloc[:,1:].mean(axis=1)
temp['row_var'] = temp.iloc[:,1:].var(axis=1)
temp.sort_values(by=["row_mean"])

[array([[-0.22060689+0.44369044j],
       [-0.46772424+0.27787374j],
       [-0.48293003-0.23691779j],
       [ 0.41099588-0.01491813j]]), array([[ 0.38597788-0.25081545j],
       [-0.3510545 +0.33862552j],
       [-0.21746451-0.46174122j],
       [-0.4671147 -0.26742152j]]), array([[ 0.1556111 -0.45422369j],
       [-0.63517603-0.05386305j],
       [ 0.40715401+0.2961001j ],
       [-0.02571258-0.33015969j]]), array([[-0.36164412+0.34990612j],
       [-0.48578244+0.02159393j],
       [ 0.47961319-0.17129841j],
       [-0.10627849+0.48955213j]])]


/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:36: ComplexWarning: Casting complex values to real discards the imaginary part
  return umr_sum(a, axis, dtype, out, keepdims, initial)


,partition,00,01,10,11,row_mean,row_var
5,"[[-XX, -YY, ZZ], [IZ, ZI]]",(11.199106980178723+0j),(8.46231054092952+1.46911073547181e-16j),(4.27751023313661+0j),(4.388536355720634+0j),(7.081866027491371+3.672776838679525e-17j),8.493934
2,"[[-XX, -YY], [ZZ, IZ, ZI]]",(9.790808426941744+0j),(9.126053580738798+3.5534003568146615e-17j),(8.542493219415292+0j),(8.225124433008723+0j),(8.92111991502614+8.883500892036654e-18j),0.356531
12,"[[-YY], [-XX, ZZ], [IZ, ZI]]",(15.936189575083109+0j),(14.876216107725803+1.1018330516038578e-16j),(5.726944093490629+0j),(7.2899285264700895+0j),(10.957319575692408+2.7545826290096446e-17j),20.238370
35,"[[-XX, -YY, ZZ], [IZ], [ZI]]",(17.857382833941983+0j),(12.707570001817524+2.2036661032077152e-16j),(8.281241795022925+0j),(6.529557022736739+0j),(11.343937913379794+5.509165258019288e-17j),19.210707
7,"[[-XX], [-YY, ZZ], [IZ, ZI]]",(13.202808239211333+0j),(12.167559401211848+1.1018330516038576e-16j),(10.349716272912481+0j),(12.281450992468429+0j),(12.000383726451023+2.754582629009644e-17j),1.069369
11,"[[-XX, -YY], [ZZ], [IZ, ZI]]",(13.613111620866412+0j),(14.619973386464682+5.3301005352219925e-17j),(10.899781565571512+0j),(12.31074125735526+0j),(12.860901957564467+1.3325251338054981e-17j),1.952205
4,"[[-XX], [-YY], [ZZ, IZ, ZI]]",(14.049825501992592+0j),(13.554248526647541+2.665050267610996e-17j),(12.194046554686278+0j),(12.676603513514237+0j),(13.118681024210161+6.66262566902749e-18j),0.526783
31,"[[-XX, -YY], [IZ], [ZZ, ZI]]",(15.175401804105434+0j),(14.304599715416998+5.3301005352219925e-17j),(13.097536543735764+0j),(12.409641868824487+0j),(13.74679498302067+1.3325251338054981e-17j),1.140395
21,"[[-XX, -YY], [ZZ, IZ], [ZI]]",(15.241367184521401+0j),(14.032662423002373+5.3301005352219925e-17j),(14.345937741594703+0j),(12.132291016355431+0j),(13.938064591368477+1.3325251338054981e-17j),1.283681
48,"[[-YY], [-XX, ZZ], [IZ], [ZI]]",(22.65988258500935+0j),(19.853760397532064+1.46911073547181e-16j),(10.12256071841152+0j),(9.648908020834504+0j),(15.57127793044686+3.672776838679525e-17j),33.337739
